In [1]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/msawant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/msawant/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix,f1_score, precision_score, recall_score

In [4]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
#file_path = "EXIST_2023/disagreement_records_task1.xlsx"
file_path="EXIST_2023/EXIST2023_Task1_en.xlsx"
data = pd.read_excel(file_path)

In [6]:
len(data)

3260

In [7]:
import re

def remove_urls_and_lower(text):
    # Define the regex pattern for URLs starting with http or https
    url_pattern = re.compile(r'http[s]?://\S+')
    # Substitute the URLs with an empty string
    cleaned_text = url_pattern.sub('', text)
    cleaned_text = cleaned_text.lower()
    return cleaned_text.strip()

In [8]:
def clean_text(text):
    return re.sub(r'[^A-Za-z\s]', '', text) if isinstance(text, str) else text
    #return re.sub(r'[^A-Za-z0-9\s]', '', text) if isinstance(text, str) else text

In [9]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
#additional_stopwords={'ago','also','along','always','amp','like','look','know','even','want','say','get','far','use','take','never','great','whole','know','even','use','take','would'}
stop_words.update(['ago','also','along','always','amp','like','look','know','even','want','say','get','far','use','take','never','great','whole','would','ok','dont','cant','shes','theyre','without','youre','isnt','us','yet','yall','u','id'])


#stop_words.update(['ago','also','along','always','amp','like','look','know','even','want','say','get','far','use','take','never','great','whole','would','ok','dont','cant','shes','theyre','without','youre'])

def stopwords_removal(text):
    words=word_tokenize(text)
    filtered_words=[w for w in words if w.lower() not in stop_words]
    return ' '.join(filtered_words)

In [10]:
from nltk.corpus import wordnet
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [11]:
from nltk.stem import WordNetLemmatizer
def apply_lemmatization(text):
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))  
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
        # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    lemmatized_sentence = " ".join(lemmatized_sentence)
    return lemmatized_sentence

In [12]:
tweet_processed=data["tweet"].apply(remove_urls_and_lower)
tweet_processed=tweet_processed.apply(clean_text)
tweet_processed=tweet_processed.apply(apply_lemmatization)
tweet_processed=tweet_processed.apply(stopwords_removal)

In [13]:
tweet_processed

0                 ffs lay blame bastard murder novel idea
1       write uni essay local pub coffee random old ma...
2       universalorl appreciate two ride team member b...
3       gmb unacceptable title men interview fact seni...
4       make hard target basically boil make sure targ...
                              ...                        
3255    idk bitch think half hang cute whore pull dirt...
3256    part experiment wordcloudsbot im learn though ...
3257    already ta readyim drip desperately rub bulge ...
3258                             clintneedcoffee whore lh
3259                             ik mandy whore cute fuck
Name: tweet, Length: 3260, dtype: object

In [14]:
vec=TfidfVectorizer(tokenizer=word_tokenize,token_pattern=None,ngram_range=(1,1))
tweet_vectorized=vec.fit_transform(tweet_processed)

Train Model for Majority vote

In [15]:
X_train, X_test, y_train, y_test = train_test_split(tweet_vectorized,data["majority_vote"], test_size=0.30, random_state=0)

In [16]:
rf_model=RandomForestClassifier(bootstrap=False, min_samples_leaf=4,min_samples_split=10,n_estimators=100,n_jobs=-1,random_state=10)
rf_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, min_samples_leaf=4,
                       min_samples_split=10, n_jobs=-1, random_state=10)

In [17]:
y_pred_rf =rf_model.predict(X_test)
y_pred_rf = y_pred_rf.astype(int)
accuracy= accuracy_score(y_test,y_pred_rf)
print("ACCURACY OF THE MODEL:", metrics.accuracy_score(y_test, y_pred_rf))
f1=f1_score(y_test,y_pred_rf,average='macro')
print(f"F1 score:{f1}")

# Calculate Precision and Recall
precision = precision_score(y_test, y_pred_rf, average='macro')
recall = recall_score(y_test, y_pred_rf, average='macro')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

ACCURACY OF THE MODEL: 0.7576687116564417
F1 score:0.7567835568198631
Precision: 0.7572300501059286
Recall: 0.7565302144249513


In [18]:
target_names = ['Yes','No']
print(classification_report(y_test, y_pred_rf, target_names=target_names))

              precision    recall  f1-score   support

         Yes       0.76      0.78      0.77       513
          No       0.75      0.73      0.74       465

    accuracy                           0.76       978
   macro avg       0.76      0.76      0.76       978
weighted avg       0.76      0.76      0.76       978



Trained Model for Female Vote

In [19]:
X_train, X_test, y_train, y_test = train_test_split(tweet_vectorized,data["female_vote"], test_size=0.30, random_state=0)

In [20]:
rf_model=RandomForestClassifier(bootstrap=False, min_samples_leaf=4,min_samples_split=10,n_estimators=100,n_jobs=-1,random_state=10)
rf_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, min_samples_leaf=4,
                       min_samples_split=10, n_jobs=-1, random_state=10)

In [21]:
y_pred_rf =rf_model.predict(X_test)
y_pred_rf = y_pred_rf.astype(int)
accuracy= accuracy_score(y_test,y_pred_rf)
print("ACCURACY OF THE MODEL:", metrics.accuracy_score(y_test, y_pred_rf))
f1=f1_score(y_test,y_pred_rf,average='macro')
print(f"F1 score:{f1}")

# Calculate Precision and Recall
precision = precision_score(y_test, y_pred_rf, average='macro')
recall = recall_score(y_test, y_pred_rf, average='macro')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

ACCURACY OF THE MODEL: 0.7269938650306749
F1 score:0.7141900211791361
Precision: 0.7183107052745404
Recall: 0.7118703358208955


In [22]:
target_names = ['Yes','No']
print(classification_report(y_test, y_pred_rf, target_names=target_names))

              precision    recall  f1-score   support

         Yes       0.75      0.80      0.77       576
          No       0.68      0.63      0.65       402

    accuracy                           0.73       978
   macro avg       0.72      0.71      0.71       978
weighted avg       0.72      0.73      0.72       978



Trained Model for Male Vote

In [23]:
X_train, X_test, y_train, y_test = train_test_split(tweet_vectorized,data["male_vote"], test_size=0.30, random_state=0)

In [24]:
rf_model=RandomForestClassifier(bootstrap=False, min_samples_leaf=4,min_samples_split=10,n_estimators=100,n_jobs=-1,random_state=10)
rf_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, min_samples_leaf=4,
                       min_samples_split=10, n_jobs=-1, random_state=10)

In [25]:
y_pred_rf =rf_model.predict(X_test)
y_pred_rf = y_pred_rf.astype(int)
accuracy= accuracy_score(y_test,y_pred_rf)
print("ACCURACY OF THE MODEL:", metrics.accuracy_score(y_test, y_pred_rf))
f1=f1_score(y_test,y_pred_rf,average='macro')
print(f"F1 score:{f1}")

# Calculate Precision and Recall
precision = precision_score(y_test, y_pred_rf, average='macro')
recall = recall_score(y_test, y_pred_rf, average='macro')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

ACCURACY OF THE MODEL: 0.7321063394683026
F1 score:0.7120685005393743
Precision: 0.7252020261514902
Recall: 0.7075406561434744


In [26]:
target_names = ['Yes','No']
print(classification_report(y_test, y_pred_rf, target_names=target_names))

              precision    recall  f1-score   support

         Yes       0.75      0.84      0.79       583
          No       0.70      0.58      0.64       395

    accuracy                           0.73       978
   macro avg       0.73      0.71      0.71       978
weighted avg       0.73      0.73      0.73       978

